In [11]:
%%writefile ocloud-update.py

# %load ocloud-update.py

# 라이브러리 로딩

from pandas_ods_reader import read_ods
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

import argparse
import pandas as pd
import pprint
import sys
import time

# 로그인
def login(driver, userid, passwd):
    # Ocloud admin 로그인 페이지로 이동
    driver.get('http://10.12.11.20:58080/admin')
    time.sleep(3)

    # UserID 입력
    username = driver.find_element_by_id("id_username")
    username.clear()
    username.send_keys(userid)

    # 암호 입력
    password = driver.find_element_by_id("id_password")
    password.clear()
    password.send_keys(passwd)

    # Login 버튼 클릭
    login = driver.find_element_by_xpath('//*[@id="login-form"]/div[3]/input').click()
    return login

# 클러스터별 VM 상세 진입

# 데이터 로딩
# def read_fn(config):
#     # 현재 위치에서 ods 파일 읽어 오기
#     # base_path = "./instance-owner-template.ods"
#     base_path = config.ods_fn
#     sheet = config.ods_sheet # "Sheet1"
#     df = read_ods(base_path, sheet, headers=True, columns=["cluster", "instance", "owner", "cellphone", "dept", "service"])
#     return df

def convert_string(x):
    if x == '신운영계':
        return 'tmslocalprd'
    elif x == '구운영계':
        return 'tmslocalover'
    
def read_fn(config):
    base_path = config.ods_fn
    sheet = config.ods_sheet # "Sheet1"    
    
    df = read_ods(base_path, sheet, headers=True) #, columns=["cluster", "instance", "owner", "cellphone", "dept", "service"])
    df['cluster'] = df.DIV.apply(convert_string)
    df2 = df[['cluster', 'hostname', '담당자', '담당팀']]
    d = { 
        "cluster" : "cluster", 
        "hostname" : "instance", 
        "담당자" : "owner", 
        "담당팀" : "dept" }
    df2.rename(columns= d, inplace = True)
    return df2[:-6]

# 인스턴스 이름으로 찾기
def find_instance(driver, instance):
    searchbar = driver.find_element_by_id("searchbar")
    searchbar.clear()
    searchbar.send_keys(instance)
    driver.find_element_by_xpath('//*[@id="changelist-search"]/div/input[2]').click()
    time.sleep(3)
    

# Contact 정보 채우기
def fill_owner_info(driver, owner, description):
    instance_owner = driver.find_element_by_id("id_form-0-owner")
    instance_owner.clear()
    instance_owner.send_keys(owner)
    
    instance_description = driver.find_element_by_id("id_form-0-description")
    instance_description.clear()
    instance_description.send_keys(description)

# 저장하기
def save(driver):
    driver.find_element_by_xpath('//*[@id="changelist-form"]/p/input').click()

def gotocluster(driver, cluster, instance, owner, description):
    print ( "시작 : " + cluster + ',' + instance + ',' + owner + ',' + description )

    if cluster == 'bigdata':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[1]/table/tbody/tr[4]/td[2]/a').click()
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
        
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster =='bigdata2':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[2]/table/tbody/tr[3]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster == 'tmscnover':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[9]/table/tbody/tr[4]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)

    elif cluster == 'tmslocalprd':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[9]/table/tbody/tr[4]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster == 'tmslocalover':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[9]/table/tbody/tr[4]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
                
    # 추가 클러스터에 적용하려면... 아래 조건문 사용하여 이동

#     elif cluster =='hkmc':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[3]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hkmcdev':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[4]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hmg':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[5]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hmg2':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[6]/table/tbody/tr[3]/td[2]/a').click()
#     elif cluster =='rnd' :
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[8]/table/tbody/tr[6]/td[2]/a').click()

    # 클러스터 이름이 맞지 않는 리스트
    else:
        print (cluster, instance)

def main(config):
        
    def print_config(config):
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(vars(config))
    print_config(config)

    config = define_argparser()
    df = read_fn(config)
    print(df)

    # 파일에서 읽은 정보 List 변수에 넣기
    clusters = df['cluster'].tolist()
    instances = df['instance'].tolist()
    owners = df['owner'].tolist()
#     cellphones = df['cellphone'].tolist()
    depts = df['dept'].tolist()
#     services = df['service'].tolist()

    # 웹브라우저 띄우기
    driver = webdriver.Chrome()
    time.sleep(3)

    userid = config.id
    passwd = config.pwd
    ret = login(driver, userid, passwd)
    print(ret)

    count = 0
    total = len(instances)
    for cluster, instance, owner, cellphone, dept, service in zip(clusters, instances, owners, cellphones, depts, services):
        description = ''
        if service != None:
            description = cellphone + " / " + dept + ' / ' + service
        else:
            description = cellphone + " / " + dept
            
        try:
            gotocluster(driver, cluster, instance, owner, description)
            count += 1
        except:
            print ( "실패 : " + cluster + ',' + instance + ',' + owner + ',' + description )
            continue
    print("총 갱신 레코드 / 갱신 성공(작업 수): ========== %d / %d" % (total, count) )


def define_argparser():
    p = argparse.ArgumentParser()
    
    p.add_argument(
        '--ods_fn',
        required=True,
        help='ods file name to continue.'
    )
    p.add_argument(
        '--ods_sheet',
        default = 'Sheet1',
        help='Sheet name is required'
    )
    p.add_argument(
        '--id',
        default = 'COC',
        help='Correct ID is required'
    )
    p.add_argument(
        '--pwd',
        default = '1234QWER',
        help='Correct Password is required'
    )

    config = p.parse_args()
    return config    

if __name__ == '__main__':
    config = define_argparser()
    main(config)


Overwriting ocloud-update.py


In [13]:
! python ocloud-update.py --ods_fn instance-owner-2021Oct10.ods --ods_sheet VM_list

In [ ]:
# 파일에서 읽은 정보 List 변수에 넣기
clusters = df['cluster'].tolist()
instances = df['instance'].tolist()
owners = df['owner'].tolist()
#     cellphones = df['cellphone'].tolist()
depts = df['dept'].tolist()
#     services = df['service'].tolist()

# 웹브라우저 띄우기
driver = webdriver.Chrome()
time.sleep(3)

userid = config.id
passwd = config.pwd
ret = login(driver, userid, passwd)
print(ret)

count = 0
total = len(instances)
for cluster, instance, owner, cellphone, dept, service in zip(clusters, instances, owners, cellphones, depts, services):
    description = ''
    if service != None:
        description = cellphone + " / " + dept + ' / ' + service
    else:
        description = cellphone + " / " + dept

    try:
        gotocluster(driver, cluster, instance, owner, description)
        count += 1
    except:
        print ( "실패 : " + cluster + ',' + instance + ',' + owner + ',' + description )
        continue
print("총 갱신 레코드 / 갱신 성공(작업 수): ========== %d / %d" % (total, count) )